In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt 
import numpy as np

import astropy.units as units 
from astropy.coordinates import SkyCoord 
from astropy.coordinates import Angle 
from astroquery.irsa_dust import IrsaDust

import healpy as hp 
import pandas as pd 

from dustmaps.sfd import SFDQuery 
from dustmaps.planck import PlanckQuery 
from dustmaps.bayestar import BayestarQuery
import dustmaps.bayestar
#dustmaps.bayestar.fetch()


import warnings
warnings.filterwarnings('ignore')
print('done importing')
#import getAVbest2

#import getAVbest import dustmaps.planck

#dustmaps.planck.fetch() print('done with importing')


done importing


In [2]:
name='M101'
Coord = SkyCoord.from_name(name, frame = 'icrs')
print(Coord)
ra0=Coord.ra
dec0=Coord.dec
print('done')


<SkyCoord (ICRS): (ra, dec) in deg
    (210.80242917, 54.34875)>
done


In [3]:
ra = np.arange(ra0.degree - 2., ra0.degree + 2., 0.05)
dec = np.arange(dec0.degree - 2., dec0.degree + 2., 0.05)
ra, dec = np.meshgrid(ra, dec)
coords = SkyCoord(ra*units.deg, dec*units.deg, frame='icrs')

sfd = SFDQuery()
# this conversion puts Av on the Schlafly system
Av_sfd = 2.742 * sfd(coords)

planck = PlanckQuery()
Av_planck = 3.1 * planck(coords)

In [4]:
bayestar = BayestarQuery(max_samples=1)
Av_bayestar = 2.742 * bayestar(coords)

Loading pixel_info ...
Loading samples ...
Loading best_fit ...
Replacing NaNs in reliable distance estimates ...
Sorting pixel_info ...
Extracting hp_idx_sorted and data_idx at each nside ...
  nside = 64
  nside = 128
  nside = 256
  nside = 512
  nside = 1024
t = 495.446 s
  pix_info:   6.056 s
   samples: 265.725 s
      best: 204.763 s
       nan:   5.501 s
      sort:  11.438 s
       idx:   1.962 s


In [5]:
ebv_map = hp.read_map('$SNFOLDER/github/ebv_tools/ebv_lhd.hpx.fits', verbose=False)
nside = hp.get_nside(ebv_map)
npix = hp.nside2npix(nside)
ordering = 'ring'


s_gal = coords.galactic
glon = s_gal.l.value
glat = s_gal.b.value
pix = hp.ang2pix(nside, glon, glat, lonlat=True)
ebv_los = ebv_map[pix]
Av_HI = ebv_los*2.742
print('done loading')

FileNotFoundError: [Errno 2] No such file or directory: '$SNFOLDER/github/ebv_tools/ebv_lhd.hpx.fits'

In [ ]:
maxboth=max(np.amax(Av_planck),np.amax(Av_sfd))
print(maxboth)

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=150)

for k,(Av,title) in enumerate([(Av_sfd, 'SFD'),
                               (Av_planck, 'Planck'),
                               (Av_bayestar, 'Bayestar')]):

    ax = fig.add_subplot(1,3,k+1)
    ax.imshow(
    np.sqrt(Av)[::,::-1],
    vmin=0.,
    vmax=maxboth,
    origin='lower',
    interpolation='nearest',
    cmap='binary',
    aspect='equal'
    )
    ax.axis('off')
    ax.set_title(title)

fig.subplots_adjust(wspace=0., hspace=0.)
plt.savefig(imagename, dpi=150)
